In [1]:
import os
import dotenv

dotenv.load_dotenv()

# Define your variable in Python
# load configuration from .env file
os.environ["APP_NAME"] = os.getenv("APP_NAME")
os.environ["PROJECT_NAME"] = os.getenv("PROJECT_NAME")
os.environ["GIT_REPO"] = os.getenv("GIT_REPO")
os.environ["CONTEXT_DIR"] = os.getenv("CONTEXT_DIR")
os.environ["RESOURCE_GROUP"] = os.getenv("RESOURCE_GROUP")
os.environ["PORT"] = os.getenv("PORT")
os.environ["API_KEY"] = os.getenv("API_KEY")


# Preperation

You might have to install the ibmcloud cli first on your system. This is how I did it for macOS

- brew install --cask ibm-cloud-cli
- ibmcloud plugin install code-engine

Generate an environment file `.env` based on the `.env-template` environment file.

# Login

In [2]:
!echo ${RESOURCE_GROUP}

tsuedbro


In [3]:
%%bash
# print the API_KEY
ibmcloud login --apikey ${API_KEY} -r "us-south"
ibmcloud target -g ${RESOURCE_GROUP}

API endpoint: https://cloud.ibm.com
Authenticating...
OK

Targeted account Federated Advocates - Johanna (cf4d3293c4e568223fdb1b408ffce752) <-> 2032340

Targeted region us-south


API endpoint:     https://cloud.ibm.com
Region:           us-south
User:             thomas.suedbroecker@de.ibm.com
Account:          Federated Advocates - Johanna (cf4d3293c4e568223fdb1b408ffce752) <-> 2032340
Resource group:   No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
Targeted resource group tsuedbro



API endpoint:     https://cloud.ibm.com
Region:           us-south
User:             thomas.suedbroecker@de.ibm.com
Account:          Federated Advocates - Johanna (cf4d3293c4e568223fdb1b408ffce752) <-> 2032340
Resource group:   tsuedbro


# Create a new IBM Cloud Code Engine application

In [4]:
%%bash
ibmcloud ce project create --name ${PROJECT_NAME}
# Select the project (optional, if not already selected)
ibmcloud ce project select --name ${PROJECT_NAME}

# Create the application
ibmcloud ce application create \
  --name ${APP_NAME}\
  --build-source ${GIT_REPO} \
  --build-context-dir ${CONTEXT_DIR} \
  --port ${PORT} \
  --strategy dockerfile \

FAILED
A project named 'galaxium-travel-services-tsuedbro' with id '62374b4b-9e5c-4930-bf2c-5f2e9a2ee5e6' and status 'active' already exists in region 'us-south'. Select a different name.



Selecting project 'galaxium-travel-services-tsuedbro'...
OK
Creating application 'galaxium-booking-system'...
Waiting for new application revision...
Submitting build run 'galaxium-booking-system-run-250813-143636502'...
For troubleshooting information visit: https://cloud.ibm.com/docs/codeengine?topic=codeengine-troubleshoot-build.
Run 'ibmcloud ce buildrun get -n galaxium-booking-system-run-250813-143636502' to check the build run status.
Run 'ibmcloud ce buildrun logs -f -n galaxium-booking-system-run-250813-143636502' to follow the logs of the build run.
Creating image 'private.us.icr.io/ce--cf4d3-1yyfsw1we2xf/app-galaxium-booking-system:250813-1236-j2sjp'...
Waiting for build run to complete...
Build run status: 'pending'
Build run status: 'running'
Build run completed successfully.
Run 'ibmcloud ce buildrun get -n galaxium-booking-system-run-250813-143636502' to check the build run status.
Waiting for application 'galaxium-booking-system' to become ready.
Configuration 'galaxium-

# Running all the tests

In [5]:
%%bash
source .venv/bin/activate
python run_tests.py all


Running all tests with coverage...
============================= test session starts ==============================
platform darwin -- Python 3.12.3, pytest-8.4.1, pluggy-1.6.0 -- /Users/thomassuedbroecker/Documents/tsuedbro/dev/learning-2024/galaxium-travels-infrastructure/booking_system_rest/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/thomassuedbroecker/Documents/tsuedbro/dev/learning-2024/galaxium-travels-infrastructure/booking_system_rest
configfile: pytest.ini
plugins: asyncio-1.1.0, anyio-4.10.0, cov-6.2.1, mock-3.14.1
asyncio: mode=Mode.STRICT, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collecting ... collected 33 items

tests/test_booking_system.py::TestFlightBooking::test_book_flight_success PASSED [  3%]
tests/test_booking_system.py::TestFlightBooking::test_book_flight_user_id_name_mismatch PASSED [  6%]
tests/test_booking_system.py::TestFlightBooking::test_book_flight_user_not_found PASSED [  9%]
tests/test_booking_system.p

# Trigger Redeployment

if you have changed something in git you can use this to trigger a redeployment

In [ ]:
%%bash

ibmcloud ce project select --name ${PROJECT_NAME}

# Create the application
ibmcloud ce application update \
  --name ${APP_NAME}\
  --build-source ${GIT_REPO} \
  --build-context-dir ${CONTEXT_DIR} \
  --port ${PORT} \
  --strategy dockerfile \

# Delete an IBM Cloud Code Engine application

you should do this to avoid unnecessary cost

In [ ]:
%%bash

# Delete the application (force, no prompt)
ibmcloud ce application delete --name ${APP_NAME} --force 